In [ ]:
import sys
sys.path.append('..')
import os
from wildlife_datasets.datasets import SeaTurtleID2022

folder0 = '../data/TurtlesTurtles'
folder1 = f'{folder0}/heads'
folder2 = f'{folder0}/bodies'

os.makedirs(folder0, exist_ok=True)
os.makedirs(folder1, exist_ok=True)
os.makedirs(folder2, exist_ok=True)

In [2]:
d = SeaTurtleID2022('/data/wildlife_datasets/data/SeaTurtleID2022', img_load='bbox')
d = d.get_subset(~d.df['bbox'].isnull())
d.df['width'] = d.df['bbox'].apply(lambda x: int(x[2]))
d.df['height'] = d.df['bbox'].apply(lambda x: int(x[3]))
d.df['path'] = d.df['path'].apply(lambda x: x.split('/')[-1].lower())
d.df = d.df.drop(['segmentation', 'original_split', 'bbox'], axis=1)
d.df.to_csv(f'{folder0}/metadata.csv', index=False)

In [3]:
d1 = SeaTurtleID2022('/data/wildlife_datasets/data/SeaTurtleID2022', img_load='bbox')
d2 = SeaTurtleID2022('/data/wildlife_datasets/data/SeaTurtleID2022', category_name='turtle', img_load='bbox')

d1 = d1.get_subset(~d1.df['bbox'].isnull())
d2 = d2.get_subset(d2.df['path'].isin(d1.df['path']))
d2 = d2.get_subset(d2.df.drop_duplicates(subset=['path']).index)
for col in ['path', 'identity', 'date', 'clarity']:
    if not d1.df[col].equals(d2.df[col]):
        raise Exception(f'column {col} not equal')
if not d1.df.index.equals(d2.df.index):
    raise Exception('index not equal')

In [4]:
for i in range(len(d1)):
    file_name = d1.df['path'].iloc[i].split('/')[-1].lower()
    d1[i].save(f'{folder1}/{file_name}')
    d2[i].save(f'{folder2}/{file_name}')